In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
from os.path import join

load_folder = 'c:/Users/balob/Documents/GITLAB/RB_BD/DATA/'

In [46]:
# Из базы данных OCS выгружаем TADIG

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT c.COUNTRY_NAME,n.NETWORK_NAME,t.TADIG_CODE,t.NETWORK_ID FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
'''
df_tadig = pd.read_sql_query(sql_srt, cnxn)
#df_tadig['NETWORK_ID']=df_tadig['NETWORK_ID'].astype('int')
display(df_tadig.head())

sql_srt=\
'''
SELECT
IIF(CURRENCY_ID=2,MO_CALL_RATE,MO_CALL_RATE*1.2) MOC,
IIF(CURRENCY_ID=2,MT_CALL_RATE,MT_CALL_RATE*1.2) MTC,
IIF(CURRENCY_ID=2,MO_SMS_RATE,MO_SMS_RATE*1.2) SMS,
IIF(CURRENCY_ID=2,DATA_RATE,DATA_RATE*1.2) DATA,
NETWORK_ID,SPONSOR_ID
FROM ROAMING_PLAN_RULES WHERE roaming_plan_id=268
AND end_date IS NULL AND is_discounted = 1
'''

df_ocs_p4 = pd.read_sql_query(sql_srt, cnxn)
#df_tadig['NETWORK_ID']=df_tadig['NETWORK_ID'].astype('int')
display(df_ocs_p4.head())

cnxn.close()

,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
0,Afghanistan,Afghan Wireless Communication Company,AFGAW,4.0
1,Afghanistan,Areeba/MTN,AFGAR,5.0
2,Afghanistan,Etisalat,AFG55,6.0
3,Afghanistan,Etisalat,AFGEA,6.0
4,Afghanistan,Roshan/TDCA,AFGK9,7.0


,MOC,MTC,SMS,DATA,NETWORK_ID,SPONSOR_ID
0,0.5616,0.0,0.05760,0.05616,10.0,2.0
1,0.8424,0.0,0.02160,0.00576,30.0,2.0
2,0.0792,0.0,0.01152,0.01152,45.0,2.0
3,0.0504,0.0,0.01872,0.01152,50.0,2.0
4,0.0504,0.0,0.01872,0.01152,51.0,2.0


In [15]:
# Загружаем новые тарифы Partner и P4 и Allowed list

df_partner = pd.read_csv(join(load_folder,'Partner_1_1_2019.csv'),sep=';')
print('Тарифы Partner:')
display(df_partner.head())

df_p4 = pd.read_csv(join(load_folder,'P4_1_1_2019.csv'),sep=';')
print('Тарифы P4:')
display(df_p4.head())

df_al=pd.read_csv(join(load_folder,'DMI.dmi_allowed_list_export_Tue_Dec_18_2018.csv'),usecols=['Sponsor','PLMN code'])
df_al.rename({'PLMN code':'TADIG_CODE_AL','Sponsor':'SPONSOR'},inplace=True,axis=1)
df_al = df_al[df_al.SPONSOR.isin(['Partner_Combined','P4_Combined'])]
df_al = df_al.replace({'Partner_Combined':'S1','P4_Combined':'S2'})
df_al = pd.merge(df_al,df_tadig[['TADIG_CODE','NETWORK_ID']],how='left',left_on='TADIG_CODE_AL',right_on='TADIG_CODE')
df_al.drop('TADIG_CODE',inplace=True,axis=1)
print('Allowed list:')
display(df_al.head())

Тарифы Partner:


,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB
0,ALBVF,27602.0,Vodafone AL,Albania,VF,0.0439,0.0,0.0062,0.0030
1,ANDMA,21303.0,Andorra Telecom SAU,Andorra,NaN,0.2000,0.0,0.0300,0.2000
2,AIACW,36584.0,AIACW AI,Anguilla,C&W Lime,0.1606,0.0,0.0128,0.0257
3,ATGCW,34492.0,ATGCW AG,Antigua And Barbuda,C&W Lime,0.1500,0.0,0.0120,0.0240
4,ARGTM,72207.0,TELE MOV AR,Argentina,Telefonica LATAM,0.0728,0.0,0.0240,0.0180


Тарифы P4:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,P4_CAMEL,P4_LTE
0,Albania,ALBtelecom sh.a.,ALBEM,0.390,NaN,0.040,0.039,yes,yes
1,Anguilla,Cable and Wireless (Anguilla) Limited,AIACW,0.800,NaN,0.080,0.080,yes,no
2,Antigua,Cable & Wireless Antigua and Barbuda Limited,ATGCW,0.800,NaN,0.080,0.080,yes,no
3,Armenia,Armenia Telephone Company (ArmenTel),ARM01,0.585,NaN,0.015,0.004,yes,no
4,Australia,SingTel Optus Pty Limited,AUSOP,0.055,NaN,0.008,0.008,yes,yes


Allowed list:


,SPONSOR,TADIG_CODE_AL,NETWORK_ID
0,S2,MNEMT,622.0
1,S1,MKDCC,569.0
2,S2,CZEET,234.0
3,S1,MWICT,577.0
4,S1,IDNTS,439.0


In [43]:
#Объединяем тарифы с TADIG и Allowed list

df_partner_net_id = pd.merge(df_partner,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Неизвестные TADIG в тарифах Partner с добавлением network_id:')
display(df_partner_net_id[df_partner_net_id.TADIG_CODE.isnull()])

df_p4_net_id = pd.merge(df_p4,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Неизвестные TADIG в тарифах P4 с добавлением network_id:')
display(df_p4_net_id[df_p4_net_id.TADIG_CODE.isnull()])

df_s1_s2_temp = pd.merge(df_partner_net_id[['MOC','MTC','SMS','MB','NETWORK_ID']],
                    df_p4_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','P4_CAMEL','P4_LTE']],
                    how='outer',on=['NETWORK_ID'],suffixes=('_S1','_S2')).drop_duplicates()
#print('Тарифы Partner и P4:')
#display(df_s1_s2_temp.head())

df_s1_s2 = pd.merge(df_s1_s2_temp,df_tadig,how='left',on='NETWORK_ID')
#print('Тарифы Partner и P4 с TADIG:')
#display(df_s1_s2.head())

df_s1_s2_sponsor = pd.merge(df_s1_s2,df_al,how='outer',on='NETWORK_ID')
#print('Тарифы Partner и P4 с привязкой к Allowed list:')
#display(df_s1_s2_sponsor.head())

# Результирующий файл

#f_list = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MOC_S1','MOC_S2','MTC_S1','MTC_S2','SMS_S1','SMS_S2','MB_S1','MB_S2','SPONSOR']
f_list = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE',
          'MOC_S1','MTC_S1','SMS_S1','MB_S1',
          'MOC_S2','MTC_S2','SMS_S2','MB_S2','SPONSOR','TADIG_CODE_AL','NETWORK_ID',]
df_s1_s2_sponsor = df_s1_s2_sponsor[f_list].drop_duplicates()
#df_s1_s2_sponsor.to_csv(join(load_folder,'Partner_P4_Compare_1_1_2019.csv'),index=False)
print('\nТарифы Partner и P4 с привязкой к Allowed list:')
df_s1_s2_sponsor.head()

Неизвестные TADIG в тарифах Partner с добавлением network_id:


,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
86,INDEU,40456.0,IdeaUP IN,India,IDEA,0.1,0.0,0.06,0.02,NaN,NaN,NaN,NaN
87,INDIB,40570.0,IDEA Bih IN,India,IDEA,0.1,0.0,0.06,0.02,NaN,NaN,NaN,NaN


Неизвестные TADIG в тарифах P4 с добавлением network_id:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,P4_CAMEL,P4_LTE,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
144,Mozambique,MOVITEL S.A,MOZVG,0.7,NaN,0.07,0.042,yes,no,NaN,NaN,NaN,NaN



Тарифы Partner и P4 с привязкой к Allowed list:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,MOC_S2,MTC_S2,SMS_S2,MB_S2,SPONSOR,TADIG_CODE_AL,NETWORK_ID
0,Albania,Vodafone,ALBVF,0.0439,0.0,0.0062,0.0030,NaN,NaN,NaN,NaN,S1,ALBVF,12.0
1,Andorra,Andorra Telecom S A U,ANDMA,0.2000,0.0,0.0300,0.2000,NaN,NaN,NaN,NaN,NaN,NaN,17.0
2,Anguilla,Cable and Wireless,AIACW,0.1606,0.0,0.0128,0.0257,0.8,NaN,0.08,0.08,S1,AIACW,20.0
3,Antigua and Barbuda,C & W,ATGCW,0.1500,0.0,0.0120,0.0240,0.8,NaN,0.08,0.08,S1,ATGCW,23.0
4,Argentina,Movistar/Telefonica,ARG01,0.0728,0.0,0.0240,0.0180,NaN,NaN,NaN,NaN,S1,ARG01,27.0


In [47]:
# Проверка наличия различных тарифов для одного и того же оператора.
# Операторы определяются по данным OCS

df = df_p4_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s1 = df[df.values > 1]
display(s1)
#df_s1_s2_sponsor[df_s1_s2_sponsor.NETWORK_ID.isin(s1.index.values)]

,MOC,MTC,SMS,MB
NETWORK_ID,,,,
412.0,2,0,1,1


In [80]:
# Проверить изменения тарифов

df=pd.merge(df_s1_s2_sponsor,\
            df_ocs_p4,how='outer',on='NETWORK_ID')

# Проверить, есть ли новые PLMN

print("Новые TADIG:")
display(df[(df.MB_S2.notnull()) & (df.SPONSOR_ID.isnull())])

# Проверить, не закрылись ли какие-нибудь сети

print("\nПропавшие TADIG:")
display(df[(df.MB_S2.isnull()) & (df.SPONSOR_ID.notnull())])

print("\nПропавшие TADIG:")
s1 = df[(df.MB_S2.isnull()) & (df.SPONSOR_ID.notnull())].NETWORK_ID.values
display(df_tadig[df_tadig.NETWORK_ID.isin(s1)])

Новые TADIG:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,MOC_S2,MTC_S2,SMS_S2,MB_S2,SPONSOR,TADIG_CODE_AL,NETWORK_ID,MOC,MTC,SMS,DATA,SPONSOR_ID
2,Anguilla,Cable and Wireless,AIACW,0.1606,0.0,0.0128,0.0257,0.800,NaN,0.080,0.0800,S1,AIACW,20.0,NaN,NaN,NaN,NaN,NaN
3,Antigua and Barbuda,C & W,ATGCW,0.1500,0.0,0.0120,0.0240,0.800,NaN,0.080,0.0800,S1,ATGCW,23.0,NaN,NaN,NaN,NaN,NaN
17,Azerbaijan,Azercell Telekom B.M.,AZEAC,0.2000,0.0,0.0200,0.0100,0.420,NaN,0.070,0.0112,S1,AZEAC,55.0,NaN,NaN,NaN,NaN,NaN
18,Barbados,C & W BET Ltd.,BRBCW,0.1500,0.0,0.0120,0.0240,0.800,NaN,0.080,0.0800,S1,BRBCW,70.0,NaN,NaN,NaN,NaN,NaN
27,British Virgin Islands,C&W LIME,VGBCW,0.1579,0.0,0.0126,0.0252,0.800,NaN,0.080,0.0800,S1,VGBCW,122.0,NaN,NaN,NaN,NaN,NaN
31,Cambodia,Viettel/Metphone Cambodia Pte Ltd,KHMVC,0.0600,0.0,0.0400,0.0200,0.700,NaN,0.070,0.0420,S1,KHMVC,139.0,NaN,NaN,NaN,NaN,NaN
32,Cambodia,Viettel/Metphone Cambodia Pte Ltd,KHMVT,0.0600,0.0,0.0400,0.0200,0.700,NaN,0.070,0.0420,S1,KHMVC,139.0,NaN,NaN,NaN,NaN,NaN
38,Cayman Islands,LIME / Cable & Wirel.,CYMCW,0.1500,0.0,0.0120,0.0240,0.800,NaN,0.080,0.0800,S1,CYMCW,174.0,NaN,NaN,NaN,NaN,NaN
40,Chile,Telefonica Movistar,CHLB2,0.0600,0.0,0.0240,0.0180,NaN,NaN,0.143,0.0400,S1,CHLB2,191.0,NaN,NaN,NaN,NaN,NaN
41,Chile,Telefonica Movistar,CHLTM,0.0600,0.0,0.0240,0.0180,NaN,NaN,0.143,0.0400,S1,CHLB2,191.0,NaN,NaN,NaN,NaN,NaN



Пропавшие TADIG:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,MOC_S2,MTC_S2,SMS_S2,MB_S2,SPONSOR,TADIG_CODE_AL,NETWORK_ID,MOC,MTC,SMS,DATA,SPONSOR_ID
88,Hong Kong (China),H3G/Hutchinson,HKGH3,0.0480,0.00,0.0240,0.0019,NaN,NaN,NaN,NaN,S1,HKGH3,392.0,0.08000,0.0,0.04500,0.04500,2.0
89,Hong Kong (China),H3G/Hutchinson,HKGHT,0.0480,0.00,0.0240,0.0019,NaN,NaN,NaN,NaN,S1,HKGH3,392.0,0.08000,0.0,0.04500,0.04500,2.0
202,Montenegro,Telenor d o o Podgorica,MNEPM,0.1004,0.00,0.0251,0.0251,NaN,NaN,NaN,NaN,S2,MNEPM,623.0,0.74880,0.0,0.04176,0.02160,2.0
203,Montenegro,Telenor d o o Podgorica,YUGPM,0.1004,0.00,0.0251,0.0251,NaN,NaN,NaN,NaN,S2,MNEPM,623.0,0.74880,0.0,0.04176,0.02160,2.0
208,Myanmar (Burma),Telenor Myanmar Limited,MMRTN,0.0350,0.08,0.0800,0.0800,NaN,NaN,NaN,NaN,S1,MMRTN,633.0,0.70000,0.0,0.05000,0.03000,2.0
261,Tajikistan,Tacom LLC,TJK91,0.0250,0.00,0.0500,0.0050,NaN,NaN,NaN,NaN,S2,TJK91,936.0,0.84240,0.0,0.02160,0.00576,2.0
271,Thailand,dtac TriNet Co Ltd,THADT,0.0800,0.03,0.0500,0.0055,NaN,NaN,NaN,NaN,S1,THADT,956.0,0.74880,0.0,0.04176,0.02160,2.0
321,French Guiana,Bouygues/DigiCel,FRAF4,NaN,NaN,NaN,NaN,0.035,NaN,0.013,NaN,NaN,NaN,309.0,0.03500,0.0,0.01300,0.00800,2.0
367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S2,MYSMT,581.0,0.60000,0.0,0.03800,0.01900,2.0
369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S2,CZERM,235.0,0.05040,0.0,0.01872,0.01152,2.0



Пропавшие TADIG:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
42,Austria,T-Mobile/Telering,AUTMM,51.0
43,Austria,T-Mobile/Telering,AUTTR,51.0
164,Croatia,Hrvatski Telekom d d,HRVCN,224.0
174,Czech Republic,T Mobile Czech Republic a s,CZERM,235.0
218,France,Free Mobile,FRAFM,296.0
223,French Guiana,Bouygues/DigiCel,FRAF4,309.0
238,Germany,E Plus Mobilfunk GmbH,DEUE1,328.0
241,Germany,T-mobile/Telekom,DEUD1,336.0
282,Hong Kong (China),H3G/Hutchinson,HKGH3,392.0
283,Hong Kong (China),H3G/Hutchinson,HKGHT,392.0


In [68]:
df_s1_s2_sponsor[df_s1_s2_sponsor.TADIG_CODE.str.startswith('FRAF4',na=False)]

,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,MOC_S2,MTC_S2,SMS_S2,MB_S2,SPONSOR,TADIG_CODE_AL,NETWORK_ID
323,French Guiana,Bouygues/DigiCel,FRAF4,NaN,NaN,NaN,NaN,0.035,NaN,0.013,NaN,NaN,NaN,309.0


In [70]:
df_p4[df_p4.TADIG=='FRAF4']

,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,P4_CAMEL,P4_LTE
69,French Guiana,Digicel Antilles Francaises Guyane,FRAF4,0.035,NaN,0.013,NaN,no,no
78,Guadeloupe,Digicel Antilles Francaises Guyane,FRAF4,0.035,NaN,0.013,NaN,no,no
138,Martinique,Digicel Antilles Francaises Guyane,FRAF4,0.035,NaN,0.013,NaN,no,no


In [67]:
df[(df.MB_S2.isnull()) & (df.SPONSOR_ID.notnull())].to_csv(join(load_folder,'result.csv'),index=False)